In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import random

import nest_asyncio
nest_asyncio.apply()
import tensorflow_federated as tff

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
#from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model


2022-06-08 10:00:11.657722: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-08 10:00:11.657759: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#BATCH_SIZE = 16
NUM_CLIENTS = 10


In [5]:
df = pd.read_csv("../datasets/TON_IoT-Datasets/Train_Test_datasets/Train_Test_Network_dataset/Train_Test_Network.csv")
df.head()
reduced_df = df[["src_port", "dst_port", "label"]]
df = reduced_df

train, test = train_test_split(df, test_size=0.2)

#columns = range(3,8)
#df = dataframe.iloc[:, columns]
train.head()

,src_port,dst_port,label
309639,41228,15600,0
15051,36252,53,0
245286,26258,53,0
443296,49449,80,1
41020,34296,10502,0


In [6]:

#IID or NOIID? Random split
client_id = np.random.choice(range(NUM_CLIENTS), size=len(train), replace=True)
#df['client_id'] = client_id
#df.head()

In [5]:
#netw_ds = train.copy()
#netw_features.pop('type')
#netw_labels = netw_features.pop('label')


In [6]:
#netw_features_dict = {name: np.array(value) 
#                         for name, value in netw_features.items()}


In [7]:
#Create a dict where keys are client ids format for tff.simulation.datasets.TestClientData
train_cl_dict = {}
for id in range(NUM_CLIENTS):
    tmp_train_df = train.loc[client_id == id]
    tmp_train_dict = {name: np.array(value) 
                         for name, value in tmp_train_df.items()}
    train_cl_dict[str(id)]=tmp_train_dict.copy()

In [8]:
#Determine max_client_ds_size
max_client_ds_size = -1
for id in range(NUM_CLIENTS):
    tmp_len = len(train.loc[client_id == id])
    if(tmp_len > max_client_ds_size):
        max_client_ds_size = tmp_len

In [9]:
# Test
tmp_test_dict = {name: np.array(value)
    for name, value in test.items()}
test_cl_dict = {'0': tmp_test_dict}
test_ds_size = len(test)

In [8]:
#for name, value in df_cl_dict.items():
#  print(f"{name:19s}: {value}")
#  break

In [10]:
#netw_ds = tf.data.Dataset.from_tensor_slices((netw_features_dict, netw_labels))
#netw_ds = tf.data.Dataset.from_tensor_slices(netw_features_dict)
train_netw_fd_ds = tff.simulation.datasets.TestClientData(train_cl_dict)
test_netw_fd_ds = tff.simulation.datasets.TestClientData(test_cl_dict)




2022-06-08 10:00:54.089595: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-08 10:00:54.089629: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-08 10:00:54.089647: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (debian): /proc/driver/nvidia/version does not exist
2022-06-08 10:00:54.089892: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
# for example in netw_fd_ds:
#   for name, value in example[0].items():
#     print(f"{name:19s}: {value}")
#   break
# # SHUFFLE DATA IN BATCH SIZES
# netw_batches = netw_ds.shuffle(len(netw_ds)).batch(BATCH_SIZE)

## Test FedAVG

In [13]:
import collections
import functools

from absl import app
from absl import flags

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

import sys

sys.path.append("/home/tester/Desktop/federated/tensorflow_federated/examples/simple_fedavg")
print(sys.path)

import simple_fedavg_tff

['/home/tester/Desktop/TF/TFF', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/home/tester/venv/lib/python3.9/site-packages', '/home/tester/Desktop/federated/tensorflow_federated/examples/simple_fedavg', '/home/tester/Desktop/federated/tensorflow_federated/examples/simple_fedavg', '/home/tester/Desktop/federated/tensorflow_federated/examples/simple_fedavg']


In [14]:
# Training hyperparameters
TOTAL_ROUNDS = 50 #NUMBER OF TOTAL TRAINING ROUNDS
ROUNDS_PER_EVAL = 1 #HOW OFTEN TO EVALUATE
TRAIN_CLIENTS_PER_ROUND = 2 #HOW MANY CLIENTS TO SAMPLE PER ROUND
CLIENT_EPOCHS_PER_ROUND = 1 #NUMBER OF EPOCHS IN THE CLIENT TO TAKE PER ROUND
BATCH_SIZE = 16 #BATCH SIZE USED ON THE CLIENT
TEST_BATCH_SIZE = 128 #MINIBATCH SIZE OF TEST DATA
SERVER_LEARNING_RATE = 1.0 #SERVER LEARNING RATE
CLIENT_LEARNING_RATE = 0.1 #CLIENT LEARNING RATE


NUM_CLIENTS = 10
#BATCH_SIZE = 1

In [74]:
def evaluate(keras_model, test_dataset):
  """Evaluate the acurracy of a keras model on a test dataset."""
  metric = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
  for batch in test_dataset:
    #DONT KNOW WHAT DOES
    predictions = keras_model(batch['x'])
    metric.update_state(y_true=batch['y'], y_pred=predictions)
  return metric.result()


def get_custom_dataset():
  def element_fn(element):
    #print(element["src_port"])
    #print(type(element))
    features = tf.convert_to_tensor([element['src_port'],element['dst_port']], dtype=tf.int64)
    return collections.OrderedDict(
      # tf.expand_dims? ADD MORE COLUMNS
        x=features, y=element['label'])

  def preprocess_train_dataset(dataset):
    # Use buffer_size same as the maximum client dataset size,
    return dataset.map(element_fn).shuffle(buffer_size=max_client_ds_size).repeat(
        count=CLIENT_EPOCHS_PER_ROUND).batch(
            BATCH_SIZE, drop_remainder=False)
  def preprocess_test_dataset(dataset):
    return dataset.map(element_fn).batch(
        TEST_BATCH_SIZE, drop_remainder=False)
  netw_train = train_netw_fd_ds.preprocess(preprocess_train_dataset)
  netw_test = preprocess_test_dataset(
      test_netw_fd_ds.create_tf_dataset_from_all_clients())
  return netw_train, netw_test


def create_fedavg_model(only_digits=True):
  """The CNN model used in https://arxiv.org/abs/1602.05629.

  Args:
    only_digits: If True, uses a final layer with 10 outputs, for use with the
      digits only EMNIST dataset. If False, uses 62 outputs for the larger
      dataset.

  Returns:
    An uncompiled `tf.keras.Model`.
  """
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(2,)),
    tf.keras.layers.Dense(8),
    tf.keras.layers.Dense(8),
    #tf.keras.layers.Dense(10, kernel_initializer=initializer),
    tf.keras.layers.Softmax()
    ])


def server_optimizer_fn():
  return tf.keras.optimizers.SGD(learning_rate=SERVER_LEARNING_RATE)


def client_optimizer_fn():
  return tf.keras.optimizers.SGD(learning_rate=CLIENT_LEARNING_RATE)

In [75]:
def main(argv):
  if len(argv) > 1:
    raise app.UsageError('Too many command-line arguments.')
  # If GPU is provided, TFF will by default use the first GPU like TF. The
  # following lines will configure TFF to use multi-GPUs and distribute client
  # computation on the GPUs. Note that we put server computatoin on CPU to avoid
  # potential out of memory issue when a large number of clients is sampled per
  # round. The client devices below can be an empty list when no GPU could be
  # detected by TF.
  client_devices = tf.config.list_logical_devices('GPU')
  server_device = tf.config.list_logical_devices('CPU')[0]
  tff.backends.native.set_local_python_execution_context(
      server_tf_device=server_device, client_tf_devices=client_devices)
  train_data, test_data = get_custom_dataset()

  def tff_model_fn():
    """Constructs a fully initialized model for use in federated averaging."""
    keras_model = create_fedavg_model(only_digits=True)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
    return tff.learning.from_keras_model(
        keras_model,
        loss=loss,
        metrics=metrics,
        input_spec=train_data.element_type_structure)

  iterative_process = simple_fedavg_tff.build_federated_averaging_process(
      tff_model_fn, server_optimizer_fn, client_optimizer_fn)
  server_state = iterative_process.initialize()
  # Keras model that represents the global model we'll evaluate test data on.
  keras_model = create_fedavg_model(only_digits=True)
  for round_num in range(TOTAL_ROUNDS):
    sampled_clients = np.random.choice(
        train_data.client_ids,
        size=TRAIN_CLIENTS_PER_ROUND,
        replace=False)
    sampled_train_data = [
        train_data.create_tf_dataset_for_client(client)
        for client in sampled_clients
    ]
    server_state, train_metrics = iterative_process.next(
        server_state, sampled_train_data)
    print(f'Round {round_num}')
    print(f'\tTraining metrics: {train_metrics}')
    if round_num % ROUNDS_PER_EVAL == 0:
      server_state.model.assign_weights_to(keras_model)
      accuracy = evaluate(keras_model, test_data)
      print(f'\tValidation accuracy: {accuracy * 100.0:.2f}%')

In [76]:
    app.run(main([None]))


Round 0
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.47048593), ('loss', 1.8035384), ('num_examples', 73592), ('num_batches', 4600)])
	Validation accuracy: 53.32%
Round 1
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.56947803), ('loss', 1.7045243), ('num_examples', 73606), ('num_batches', 4601)])
	Validation accuracy: 60.24%
Round 2
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.5999673), ('loss', 1.6740227), ('num_examples', 73459), ('num_batches', 4592)])
	Validation accuracy: 60.24%
Round 3
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.60148525), ('loss', 1.6725043), ('num_examples', 73523), ('num_batches', 4596)])
	Validation accuracy: 60.26%
Round 4
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.60166067), ('loss', 1.6723248), ('num_examples', 74188), ('num_batches', 4637)])
	Validation accuracy: 60.26%
Round 5
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.6013127

FATAL Flags parsing error: Unknown command line flag 'ip'
Pass --helpshort or --helpfull to see help on flags.


AssertionError: 

In [63]:
# import nest_asyncio
# nest_asyncio.apply()

try:
    app.run(main([None]))
except:
    print("exec end error")

exec end error
